In [ ]:
import os as o

from pathlib import Path
import time
import matplotlib.pyplot as plt
import crosspy as xpy


def crop_image(img,w,h, xc, yc):
    x1 = int(xc-w/2)
    x2 = int(xc+w/2)
    y1 = int(yc-h/2)
    y2 = int(yc+h/2)
    cropped_images = img[y1:y2,x1:x2,:]
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(cropped_images[:,:,0])
    ax[1].imshow(cropped_images[:,:,1])
    print(cropped_images.shape)

    return cropped_images

folder_path=Path(r'C:\Alex\crosspy\data\ce4511_2\aligned_stiches')
Images = xpy.Imset(folder_path,'tif',[0,1])


coarse = 300#300
coarse_overlap = 70
fine = 32
overlap = 80
# # fft filter settings: high pass, high pass width, low pass, low pass width
filter_settings=[4,2,15,8]
roi_1stpass = dict(size_pass = coarse, overlap_percentage = coarse_overlap, xcf_mesh=coarse)
ims = Images.imload([0,1])
print(ims.shape)

#cropping
# cropped = crop_image(ims, 6000,4000, 6000, 4000)
#cropped2 = crop_image(cropped, 3000, 2000, 6000-1500, 4000-1000)


In [ ]:
coarse = 200
coarse_overlap = 50
fine = 48
overlap = 80
# # fft filter settings: high pass, high pass width, low pass, low pass width
filter_settings=[4,2,15,8]
roi_1stpass = dict(size_pass = coarse, overlap_percentage = coarse_overlap, xcf_mesh=coarse)
dic_1stpass=xpy.DIC(cropped,roi_1stpass,filter_settings)


In [ ]:
if __name__ == "__main__":

    t0=time.time()

    # # build the dic class (but don't run it yet):
    
    # # run the dic on specified images within the stack, and get displacements:
    dic_1stpass.run_sequential(cores=40)
    dic_1stpass.plot_displacements()

    # # correct the images and instantiate a new DIC class
    #corrected_images=dic_1stpass.correct(method='rigid',printing=1)

    #dic_1stpass=xpy.DIC(corrected_images,roi_1stpass,filter_settings)

    #dic_1stpass.run_sequential(cores=20)

    corrected_images=dic_1stpass.correct(method='polynomial',printing=1)
    #cropped2 = crop_image(corrected_images, 3000, 2000, 6000-1500, 4000-1000)
    roi_2ndpass = dict(size_pass = fine, overlap_percentage = overlap, xcf_mesh=fine)

    dic_2ndpass = xpy.DIC(corrected_images,roi_2ndpass,filter_settings,savingfolder=dic_1stpass.folder)
    print(dic_2ndpass.n_subsets)
    # # run the second pass
    dic_2ndpass.run_sequential(cores=40)#, hs=False)
    dic_2ndpass.plot_displacements()

    dic_2ndpass.calculate_strain()
    dic_2ndpass.plot_strains()

    #dic_2ndpass.save_data()

In [ ]:

import matplotlib.pyplot as plt 
import cmocean
from matplotlib import cm
roi_size = fine
roi_overlap = overlap
strain_comp = 11


mapmo = 0

eff = dic_2ndpass.strain_eff[100:,:,mapmo]
e11 = dic_2ndpass.strain_11[:,:,mapmo]
e22 = dic_2ndpass.strain_22[:,:,mapmo]
e12 = dic_2ndpass.strain_12[:,:,mapmo]
cc = dic_2ndpass.ph_maps[:,:,mapmo]
# j_map = dic_2ndpass.js_maps[:,:,0]

# fig,axs = plt.subplots(1,2, figsize=(30,10))


# im1 = axs[0].imshow(e11, vmin=-0.02,vmax=0.02, cmap=cmocean.cm.deep_r)
# im2 = axs[1].imshow(j_map,vmin=1, vmax=10, cmap=cmocean.cm.deep_r)


# fig.colorbar(im1, ax=axs[0], shrink=0.8)
# fig.colorbar(im2, ax=axs[1], shrink=0.8)


fig,ax = plt.subplots()


im1 = ax.imshow(e11, vmin=-0.02,vmax=0.02, cmap=cmocean.cm.deep_r)

fig.colorbar(im1, ax=ax, shrink=0.8)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

print(eff[100:,:].mean())

#fid = "ce4522_nohs{}_size{}_overlap{}_coarse{}".format(str("eff"),str(roi_size),str(roi_overlap),str(coarse))
fid = "NonHS_sz{},st{},csz{},csst{}".format(str(roi_size),str(roi_overlap),str(coarse),str(coarse_overlap))
fid = o.path.join(folder_path, fid + "." + "tif")

plt.savefig(fid, dpi=400)

fig,ax = plt.subplots()


im2 = ax.imshow(cc,vmin=0,vmax=2.5e9, cmap=cmocean.cm.deep_r)

fig.colorbar(im2, ax=ax, shrink=0.8)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

print(eff[100:,:].mean())

#fid = "ce4522_nohs{}_size{}_overlap{}_coarse{}".format(str("eff"),str(roi_size),str(roi_overlap),str(coarse))
fid = "NonHSph_sz{},st{},csz{},csst{}".format(str(roi_size),str(roi_overlap),str(coarse),str(coarse_overlap))
fid = o.path.join(folder_path, fid + "." + "tif")

plt.savefig(fid, dpi=400)

In [ ]:

w = 12000 - 6000
h = 8000 - 4000
xc = 6000
yc = 4000
x1 = int(xc-w/2)
x2 = int(xc+w/2)
y1 = int(yc-h/2)
y2 = int(yc+h/2)
cropped2 = crop_image(corrected_images, 3000, 2000, 6000-1500, 4000-1000)
plt.imshow(cropped2[:,:,1])
print(cropped2.shape)

double_correct = True


In [ ]:
# Attempt to double correct around fine area
if __name__ == "__main__":

    roi_doubc = dict(size_pass = 200, overlap_percentage = 50, xcf_mesh=250)
    dic_doublecorr = xpy.DIC(cropped2,roi_doubc,filter_settings,savingfolder=dic_1stpass.folder)
    dic_doublecorr.run_sequential(cores=20)
    corrected_ims2 = dic_doublecorr.correct(printing=1)

    hs_size = 24
    hs_overlap = 80

    roi_3rdpass = dict(size_pass = hs_size, overlap_percentage = hs_overlap, xcf_mesh=200)

    dic_3rdpass = xpy.DIC(corrected_ims2,roi_3rdpass,filter_settings,savingfolder=dic_1stpass.folder)


    print(dic_3rdpass.n_subsets)
    # # run the second pass
    dic_3rdpass.run_sequential(cores=40, hs=False, cc_t=0.)
    dic_3rdpass.plot_displacements()

    

In [ ]:
if __name__ == "__main__":

    hs_size = 48
    hs_overlap = 80

    roi_3rdpass = dict(size_pass = hs_size, overlap_percentage = hs_overlap, xcf_mesh=hs_size)

    dic_3rdpass = xpy.DIC(corrected_images,roi_3rdpass,filter_settings,savingfolder=dic_1stpass.folder)


    print(dic_3rdpass.n_subsets)
    # # run the second pass
    dic_3rdpass.run_sequential(cores=40, hs=True, cc_t=0.)
    dic_3rdpass.plot_displacements()

In [ ]:
if __name__ == "__main__":
    dic_3rdpass.calculate_strain()
    dic_3rdpass.plot_strains()

In [ ]:
eff3 = dic_3rdpass.strain_eff[:,:,0]
e113 = dic_3rdpass.strain_11[:,:,0]
e223 = dic_3rdpass.strain_22[:,:,0]
e123 = dic_3rdpass.strain_12[:,:,0]

j_map = j_map#dic_3rdpass.js_maps[:,:,0]

fig,ax = plt.subplots()
ph_map = dic_3rdpass.ph_maps[:,:,0]

im1 = ax.imshow(e113, vmin=-0.02,vmax=0.02, cmap=cmocean.cm.deep_r)

fig.colorbar(im1, ax=ax, shrink=0.8)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

fid = "SMALL_nHS_xx{},st{},csz{},csst{}".format(str(hs_size),str(hs_overlap),str(coarse),str(coarse_overlap))
fid = o.path.join(folder_path, fid + "." + "tif")

plt.savefig(fid, dpi=400)

fig,ax = plt.subplots()

im2 = ax.imshow(ph_map,vmin=0, cmap=cmocean.cm.deep_r)

fig.colorbar(im2, ax=ax, shrink=0.8)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

fid = "SMALL_nHS_ph{},st{},csz{},csst{}".format(str(hs_size),str(hs_overlap),str(coarse),str(coarse_overlap))
fid = o.path.join(folder_path, fid + "." + "tif")

plt.savefig(fid, dpi=400)


In [ ]:
## Lineplots
import numpy as np

print(e113.shape)
# horizontal
xs = 100
xe = 400
yfixed = 160

y = e113[yfixed,xs:xe]

x = np.arange(0,y.shape[0])*30/1000
print(np.mean(e113))

plt.subplot
plt.plot(x,y)
plt.ylim([-0.08,0.08])

In [ ]:
fig, ax = plt.subplots()

im3 = plt.imshow(j_map,vmin=1,vmax=10)
fig.colorbar(im3, ax=ax shrink=0.8)